In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

(beta) Building a Convolution/Batch Norm fuser in FX
====================================================

**Author**: [Horace He](https://github.com/chillee)

In this tutorial, we are going to use FX, a toolkit for composable
function transformations of PyTorch, to do the following:

1)  Find patterns of conv/batch norm in the data dependencies.
2)  For the patterns found in 1), fold the batch norm statistics into
    the convolution weights.

Note that this optimization only works for models in inference mode
(i.e. [mode.eval()]{.title-ref})

We will be building the fuser that exists here:
<https://github.com/pytorch/pytorch/blob/orig/release/1.8/torch/fx/experimental/fuser.py>


First, let\'s get some imports out of the way (we will be using all of
these later in the code).


For this tutorial, we are going to create a model consisting of
convolutions and batch norms. Note that this model has some tricky
components - some of the conv/batch norm patterns are hidden within
Sequentials and one of the `BatchNorms` is wrapped in another Module.


Fusing Convolution with Batch Norm
==================================

One of the primary challenges with trying to automatically fuse
convolution and batch norm in PyTorch is that PyTorch does not provide
an easy way of accessing the computational graph. FX resolves this
problem by symbolically tracing the actual operations called, so that we
can track the computations through the [forward]{.title-ref} call,
nested within Sequential modules, or wrapped in an user-defined module.


This gives us a graph representation of our model. Note that both the
modules hidden within the sequential as well as the wrapped Module have
been inlined into the graph. This is the default level of abstraction,
but it can be configured by the pass writer. More information can be
found at the FX overview
<https://pytorch.org/docs/master/fx.html#module-torch.fx>


Fusing Convolution with Batch Norm
==================================

Unlike some other fusions, fusion of convolution with batch norm does
not require any new operators. Instead, as batch norm during inference
consists of a pointwise add and multiply, these operations can be
\"baked\" into the preceding convolution\'s weights. This allows us to
remove the batch norm entirely from our model! Read
<https://nenadmarkus.com/p/fusing-batchnorm-and-conv/> for further
details. The code here is copied from
<https://github.com/pytorch/pytorch/blob/orig/release/1.8/torch/nn/utils/fusion.py>
clarity purposes.


FX Fusion Pass
==============

Now that we have our computational graph as well as a method for fusing
convolution and batch norm, all that remains is to iterate over the FX
graph and apply the desired fusions.


```{=html}
<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
```
```{=html}
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
```
```{=html}
<p>We make some simplifications here for demonstration purposes, such as onlymatching 2D convolutions. View<a href="https://github.com/pytorch/pytorch/blob/master/torch/fx/experimental/fuser.py">https://github.com/pytorch/pytorch/blob/master/torch/fx/experimental/fuser.py</a>for a more usable pass.</p>
```
```{=html}
</div>
```


Testing out our Fusion Pass
===========================

We can now run this fusion pass on our initial toy model and verify that
our results are identical. In addition, we can print out the code for
our fused model and verify that there are no more batch norms.


Benchmarking our Fusion on ResNet18
===================================

We can test our fusion pass on a larger model like ResNet18 and see how
much this pass improves inference performance.


As we previously saw, the output of our FX transformation is
(\"torchscriptable\") PyTorch code, we can easily `jit.script` the
output to try and increase our performance even more. In this way, our
FX model transformation composes with TorchScript with no issues.


In [ ]:
############
# Conclusion
# ----------
# As we can see, using FX we can easily write static graph transformations on
# PyTorch code.
#
# Since FX is still in beta, we would be happy to hear any
# feedback you have about using it. Please feel free to use the
# PyTorch Forums (https://discuss.pytorch.org/) and the issue tracker
# (https://github.com/pytorch/pytorch/issues) to provide any feedback
# you might have.

# %%%%%%RUNNABLE_CODE_REMOVED%%%%%%